In [ ]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
csv_path = '/content/drive/MyDrive/ML_Dataset/AQI_2010_2023/stations_info.csv'

try:
  df = pd.read_csv(csv_path)
  print(df.head())
except FileNotFoundError:
  print(f"Error: The file '{csv_path}' was not found. Make sure the file path is correct.")
except Exception as e:
  print(f"An error occurred: {e}")